In [ ]:
%matplotlib inline
from __future__ import division, print_function
import sys, os
import numpy as np
from astropy.table import Table
import fitsio
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as u
import healpy as hp

from desitarget.targets import desi_mask
from match_coord import search_around, scatter_plot, match_coord

In [ ]:
def relative_density_plot(d_ra, d_dec, d2d, search_radius, ref_density, nbins=101):
    bins = np.linspace(-search_radius, search_radius, nbins)
    bin_spacing = bins[1] - bins[0]
    bincenter = (bins[1:]+bins[:-1])/2
    mesh_ra, mesh_dec = np.meshgrid(bincenter, bincenter)
    mesh_d2d = np.sqrt(mesh_ra**2 + mesh_dec**2)
    mask = (d2d>2.)
    density, _, _ = np.histogram2d(d_ra[mask], d_dec[mask], bins=bins)/(bin_spacing**2)
    mask = mesh_d2d >= bins.max()-bin_spacing
    density[mask] = np.nan
    density_ratio = density/ref_density
    plt.figure(figsize=(8, 8))
    plt.imshow(density_ratio.transpose()-1, origin='lower', aspect='equal', 
               cmap='seismic', extent=bins.max()*np.array([-1, 1, -1, 1]), vmin=-3, vmax=3)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.show()

In [ ]:
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)

In [ ]:
columns = ['RA', 'DEC', 'DESI_TARGET', 'NOBS_G', 'NOBS_R', 'NOBS_Z']

# Select LRG sample from DESI target catalog
cat = fitsio.read('/project/projectdirs/desi/target/catalogs/dr7.1/0.22.0/targets-dr7.1-0.22.0.fits', columns=['DESI_TARGET'])
print(len(cat))

# Select LRGs
idx = np.where((cat["DESI_TARGET"] & desi_mask["LRG"])!=0)[0]
cat = fitsio.read('/project/projectdirs/desi/target/catalogs/dr7.1/0.22.0/targets-dr7.1-0.22.0.fits', columns=columns, rows=idx)
print(len(cat))

# Require 2+ exposures in grz
mask = (cat['NOBS_G']>=2) & (cat['NOBS_R']>=2) & (cat['NOBS_Z']>=2)
cat = cat[mask]
print(len(cat))

In [ ]:
# Load trimmed WISE bright star catalog
wisecat = Table.read('/global/homes/r/rongpu/mydesi/useful/w1_bright-13.3_trim_dr5_region.fits')
print(len(wisecat))
wisecat['w1ab'] = np.array(wisecat['W1MPRO']) + 2.7

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(cat['RA'][::5], cat['DEC'][::5], '.', markersize=0.2, alpha=0.2)
plt.plot(wisecat['RA'][::5], wisecat['DEC'][::5], '.', markersize=0.3, alpha=0.2)
plt.axis([0, 360, -23, 35])
plt.show()

In [ ]:
# Select only nearby WISE stars to speed up computation
nside = 512
pix_area = hp.pixelfunc.nside2pixarea(nside, degrees=True)
print('Pixel area:', pix_area)

pix_cat = np.unique(hp.pixelfunc.ang2pix(nside, cat['RA'], cat['DEC'], lonlat=True))
pix_star = np.unique(hp.pixelfunc.ang2pix(nside, wisecat['RA'], wisecat['DEC'], lonlat=True))

pix_cat_ra, pix_cat_dec = hp.pixelfunc.pix2ang(nside, pix_cat, lonlat=True)
pix_star_ra, pix_star_dec = hp.pixelfunc.pix2ang(nside, pix_star, lonlat=True)

pix_search_radius = 0.5*3600 # arcsec
idx_cat, idx_star, _, _, _ = match_coord(pix_cat_ra, pix_cat_dec, pix_star_ra, pix_star_dec, search_radius=pix_search_radius, plot_q=False, verbose=False, keep_all_pairs=True)
pix_star_nearby = np.unique(pix_star[idx_star])

pix_star_all = hp.pixelfunc.ang2pix(nside, wisecat['RA'], wisecat['DEC'], lonlat=True)
mask = np.in1d(pix_star_all, pix_star_nearby)

wisecat = wisecat[mask]
print(len(wisecat), np.sum(mask)/len(mask))

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(cat['RA'][::5], cat['DEC'][::5], '.', markersize=0.2, alpha=0.2)
plt.plot(wisecat['RA'][::5], wisecat['DEC'][::5], '.', markersize=0.3, alpha=0.2)
plt.axis([0, 360, -23, 35])
plt.show()

In [ ]:
# ra1 = np.array(cat['RA'])
# dec1 = np.array(cat['DEC'])
# ra2 = np.array(wisecat['RA'])
# dec2 = np.array(wisecat['DEC'])

# Convert to the ecliptic coordinates
c_decals = SkyCoord(ra=cat['RA']*u.degree, dec=cat['DEC']*u.degree, frame='icrs')
c_wise = SkyCoord(ra=wisecat['RA']*u.degree, dec=wisecat['DEC']*u.degree, frame='icrs')
temp = c_decals.barycentrictrueecliptic
ra1, dec1 = np.array(temp.lon), np.array(temp.lat)
temp = c_wise.barycentrictrueecliptic
ra2, dec2 = np.array(temp.lon), np.array(temp.lat)

In [ ]:
w1min, w1max, w1nbins = 8, 12, 5
w1mag_bins = np.linspace(w1min, w1max, w1nbins)
print(w1mag_bins)

In [ ]:
search_radius = 240.

# Paramater for estimating the overdensities
annulus_min = 180.
annulus_max = 240.

for index in range(len(w1mag_bins)):
    
    if index==0:
        mask = (wisecat['w1ab']<w1mag_bins[index])
        title = 'W1mag < {:.2f}'.format(w1mag_bins[0], np.sum(mask))
    else:
        mask = (wisecat['w1ab']>w1mag_bins[index-1]) & (wisecat['w1ab']<w1mag_bins[index])
        title = '{:.2f} < W1mag < {:.2f}'.format(w1mag_bins[index-1], w1mag_bins[index], np.sum(mask))
        
    print(title)

    idx2, idx1, d2d, d_ra, d_dec = search_around(ra2[mask], dec2[mask], ra1, dec1, search_radius=search_radius)

    markersize = np.max([0.01, np.min([10, 0.3*100000/len(idx2)])])    
    axis = [-search_radius*1.05, search_radius*1.05, -search_radius*1.05, search_radius*1.05]
    axScatter = scatter_plot(d_ra, d_dec, markersize=markersize, alpha=0.4, figsize=6.5, axis=axis, title=title)
    
    ntot_annulus = np.sum((d2d>annulus_min) & (d2d<annulus_max))
    density_annulus = ntot_annulus/(np.pi*(annulus_max**2 - annulus_min**2))
    
    relative_density_plot(d_ra, d_dec, d2d, search_radius, ref_density=density_annulus)

In [ ]:
w1min, w1max, w1nbins = 12, 16, 5
w1mag_bins = np.linspace(w1min, w1max, w1nbins)
print(w1mag_bins)

In [ ]:
# Zoom in
search_radius = 90.

# Paramater for estimating the overdensities
annulus_min = 60.
annulus_max = 90.

for index in range(1, len(w1mag_bins)):
    
    if index==0:
        mask = (wisecat['w1ab']<w1mag_bins[index])
        title = 'W1mag < {:.2f}'.format(w1mag_bins[0], np.sum(mask))
    else:
        mask = (wisecat['w1ab']>w1mag_bins[index-1]) & (wisecat['w1ab']<w1mag_bins[index])
        title = '{:.2f} < W1mag < {:.2f}'.format(w1mag_bins[index-1], w1mag_bins[index], np.sum(mask))
        
    print(title)

    idx2, idx1, d2d, d_ra, d_dec = search_around(ra2[mask], dec2[mask], ra1, dec1, search_radius=search_radius)

    markersize = np.max([0.01, np.min([10, 0.3*100000/len(idx2)])])    
    axis = [-search_radius*1.05, search_radius*1.05, -search_radius*1.05, search_radius*1.05]
    axScatter = scatter_plot(d_ra, d_dec, markersize=markersize, alpha=0.4, figsize=6.5, axis=axis, title=title)
    
    ntot_annulus = np.sum((d2d>annulus_min) & (d2d<annulus_max))
    density_annulus = ntot_annulus/(np.pi*(annulus_max**2 - annulus_min**2))
    
    relative_density_plot(d_ra, d_dec, d2d, search_radius, ref_density=density_annulus)